In [2]:
%AddDeps org.bdgenomics.adam adam-core-spark2_2.11 0.24.0
%AddDeps comp.bio.aging adam-playground_2.11 0.0.13 --repository https://dl.bintray.com/comp-bio-aging/main/

Marking org.bdgenomics.adam:adam-core-spark2_2.11:0.24.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir3070127853329215662/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir3070127853329215662/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.jar
-> New file at /tmp/toree-tmp-dir3070127853329215662/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.pom


Waiting for a Spark session to start...

Marking comp.bio.aging:adam-playground_2.11:0.0.13 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir3070127853329215662/toree_add_deps/
-> https://dl.bintray.com/comp-bio-aging/main/
-> https://repo1.maven.org/maven2


In [3]:
%AddDeps org.apache.hadoop hadoop-azure 2.7.6
%AddDeps com.microsoft.azure azure-storage 2.0.0

Marking org.apache.hadoop:hadoop-azure:2.7.6 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir3070127853329215662/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir3070127853329215662/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.jar
-> New file at /tmp/toree-tmp-dir3070127853329215662/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.pom


Waiting for a Spark session to start...

Marking com.microsoft.azure:azure-storage:2.0.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir3070127853329215662/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir3070127853329215662/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.jar
-> New file at /tmp/toree-tmp-dir3070127853329215662/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.pom


In [4]:
import  org.apache.spark._
import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._

In [5]:
def sparkHadoopConf(sc: SparkContext, acountName: String, accountKey: String) = {
  sc.hadoopConfiguration.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
  sc.hadoopConfiguration.set("fs.azure.account.key." + acountName + ".blob.core.windows.net", accountKey)
  sc
}

sparkHadoopConf: (sc: org.apache.spark.SparkContext, acountName: String, accountKey: String)org.apache.spark.SparkContext


In [6]:
def azurize(container: String, accountName: String, blobFile: String): String = "wasbs://"+container+"@"+accountName+".blob.core.windows.net/"+blobFile 

def writeText2Azure[T]( rdd: RDD[T], container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  rdd.saveAsTextFile(url)
  url
}

def writeTsv2Azure( df: DataFrame, container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  df.write.option("sep","\t").option("header","true").csv(url)
  url
}

azurize: (container: String, accountName: String, blobFile: String)String
writeText2Azure: [T](rdd: org.apache.spark.rdd.RDD[T], container: String, accountName: String, blobFile: String)String
writeTsv2Azure: (df: org.apache.spark.sql.DataFrame, container: String, accountName: String, blobFile: String)String


In [7]:
val account = "pipelines1"
val key = ""
val connString = s"DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=${key};EndpointSuffix=core.windows.net"
def az(path: String): String = azurize("storage", account, path)

account = pipelines1
key = 
connString = DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net


az: (path: String)String


DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net

In [8]:
val sparkContext = sc
sparkHadoopConf(sparkContext, account, key)
  
val spark = SparkSession
  .builder()
  .appName("mapping_models")
  .getOrCreate()

sparkContext = org.apache.spark.SparkContext@5c0329bf
spark = org.apache.spark.sql.SparkSession@12e9d200


Spark app-20180703170633-0003: Some(http://c8fc1f462c1d:4041)

In [9]:
def write(df: DataFrame, url: String, header: Boolean = true) = {
  df.coalesce(1).write.option("sep","\t").option("header", header).csv(url)
  url
}

write: (df: org.apache.spark.sql.DataFrame, url: String, header: Boolean)String


In [10]:
def readTSV(path: String, header: Boolean = false, sep: String = "\t"): DataFrame = spark.read
    .option("sep", sep)
    .option("comment", "#")
    .option("inferSchema", true)
    .option("header", header)
    .option("ignoreLeadingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("maxColumns", 150000)
    .csv(path)

readTSV: (path: String, header: Boolean, sep: String)org.apache.spark.sql.DataFrame


In [11]:
 def toVectors(dataFrame: DataFrame, columns: Seq[String], output: String) = {  
      import org.apache.spark.ml.feature.VectorAssembler
      import org.apache.spark.ml.linalg.Vectors
      val assembler = new VectorAssembler()
        .setInputCols(columns.toArray)
        .setOutputCol(output)
      assembler.transform(dataFrame.na.fill(0.0, columns).na.fill(0.0)).select(output)
    }

toVectors: (dataFrame: org.apache.spark.sql.DataFrame, columns: Seq[String], output: String)org.apache.spark.sql.DataFrame


In [27]:
def rename(dataFrame: DataFrame, renamings: Map[String, String]) =   {
      val newColumns = dataFrame.columns.map(c=> renamings.getOrElse(c, c))
      dataFrame.toDF(newColumns:_*)
    }
def renameFunction(renamings: Map[String, String]) = udf[String, String](c=> renamings.getOrElse(c, c))

rename: (dataFrame: org.apache.spark.sql.DataFrame, renamings: Map[String,String])org.apache.spark.sql.DataFrame
renameFunction: (renamings: Map[String,String])org.apache.spark.sql.expressions.UserDefinedFunction


In [22]:
def uselect(dataFrame: DataFrame, prefix: String) = {
    val liverNew = prefix + "_liver"
    val kidneyNew = prefix + "_kidney"
    dataFrame
        .withColumnRenamed("liver", liverNew)
        .withColumnRenamed("kidney", kidneyNew)
    .select("uniref90", liverNew, kidneyNew)
}

uselect: (dataFrame: org.apache.spark.sql.DataFrame, prefix: String)org.apache.spark.sql.DataFrame


In [13]:
val expressionsPath = "expressions"
val byGoPath = expressionsPath + "/go"
val comparison = byGoPath + "/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv"
val grouped = byGoPath + "/grouped"
val ranked = byGoPath + "/grouped/ranked"
val transcriptsPath = expressionsPath + "/transcripts"
val indexesPath = "indexes"
val reactomePath = indexesPath + "/reactome"
val unirefPath = expressionsPath + "/" + "uniref90"

expressionsPath = expressions
byGoPath = expressions/go
comparison = expressions/go/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv
grouped = expressions/go/grouped
ranked = expressions/go/grouped/ranked
transcriptsPath = expressions/transcripts
indexesPath = indexes
reactomePath = indexes/reactome
unirefPath = expressions/uniref90


expressions/uniref90

Merge nonmodel Uniref90
-----------------------
Uniref90 or nonmodel animals

In [16]:
import org.apache.spark.sql.functions._
import spark.implicits._
val cut_id = udf[String, String]( _.replace("UniRef90_",""))

cut_id = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [19]:
val nonModelUniref90_outer = readTSV(az(unirefPath + "/non_model_uniref90_outer.tsv"), true)

nonModelUniref90_outer = [uniref90: string, gray_whale_liver: double ... 7 more fields]


[uniref90: string, gray_whale_liver: double ... 7 more fields]

In [86]:
/*
import org.apache.spark.storage._

val mapping =  readTSV(az("/indexes/uniprot/idmapping_selected.tab"))
.toDF("uniprot_ac","uniprot_id","entrez","refSeq", "gi","pdb", "go", "uniref100", "uniref90", "uniref50", "uniparc", "pir", "taxon", "mim", "unigene", "pubmed", "embl", "embl_cds", "ensembl", "ensembl_trs", "ensembl_pro", "additional_pubmed")
.persist(StorageLevel.MEMORY_AND_DISK) 
*/

mapping = [uniprot_ac: string, uniprot_id: string ... 20 more fields]


[uniprot_ac: string, uniprot_id: string ... 20 more fields]

Merge model uniref90-s
----------------------

In [87]:
//%%dataframe mappings

An error occurred converting DataFrame to html.
<console>:93: error: not found: value mappings
       mappings
       ^


In [20]:
val human_transcripts = readTSV(az(transcriptsPath + "/human_transcripts_all.tsv"), true)
val cow_mRNA_transcripts = readTSV(az(transcriptsPath + "/cow_mRNA_transcripts_all.tsv"), true)
val cow_totalRNA_transcripts = readTSV(az(transcriptsPath + "/cow_totalRNA_transcripts_all.tsv"), true)
val mouse_mRNA_transcripts = readTSV(az(transcriptsPath + "/mouse_mRNA_transcripts_all.tsv"), true)
val mouse_totalRNA_transcripts = readTSV(az(transcriptsPath + "/mouse_totalRNA_transcripts_all.tsv"), true)

human_transcripts = [transcript: string, uniref90: string ... 13 more fields]
cow_mRNA_transcripts = [transcript: string, uniref90: string ... 13 more fields]
cow_totalRNA_transcripts = [transcript: string, uniref90: string ... 13 more fields]
mouse_mRNA_transcripts = [transcript: string, uniref90: string ... 13 more fields]
mouse_totalRNA_transcripts = [transcript: string, uniref90: string ... 13 more fields]


[transcript: string, uniref90: string ... 13 more fields]

In [23]:
val allTr = uselect(human_transcripts, "human").na.fill(0)
  .join(uselect(cow_mRNA_transcripts, "cow_mRNA"), Seq("uniref90"), "outer").na.fill(0)
  .join(uselect(cow_totalRNA_transcripts, "cow_totalRNA"), Seq("uniref90"), "outer").na.fill(0)
  .join(uselect(mouse_mRNA_transcripts, "mouse_mRNA"), Seq("uniref90"), "outer").na.fill(0)
  .join(uselect(mouse_totalRNA_transcripts, "mouse_totalRNA"), Seq("uniref90"), "outer").na.fill(0)
val allGroup =     allTr.groupBy("uniref90")

allTr = [uniref90: string, human_liver: double ... 9 more fields]
allGroup = RelationalGroupedDataset: [grouping expressions: [uniref90: string], value: [uniref90: string, human_liver: double ... 9 more fields], type: GroupBy]


RelationalGroupedDataset: [grouping expressions: [uniref90: string], value: [uniref90: string, human_liver: double ... 9 more fields], type: GroupBy]

In [24]:
val modelSum = allGroup.sum(allTr.columns.tail:_*)

modelSum = [uniref90: string, sum(human_liver): double ... 9 more fields]


[uniref90: string, sum(human_liver): double ... 9 more fields]

In [25]:
val modelRenamings = modelSum.columns.filter(_.startsWith("sum(")).map(v=>v->v.replace("sum(","").replace(")", ""))

modelRenamings = Array((sum(human_liver),human_liver), (sum(human_kidney),human_kidney), (sum(cow_mRNA_liver),cow_mRNA_liver), (sum(cow_mRNA_kidney),cow_mRNA_kidney), (sum(cow_totalRNA_liver),cow_totalRNA_liver), (sum(cow_totalRNA_kidney),cow_totalRNA_kidney), (sum(mouse_mRNA_liver),mouse_mRNA_liver), (sum(mouse_mRNA_kidney),mouse_mRNA_kidney), (sum(mouse_totalRNA_liver),mouse_totalRNA_liver), (sum(mouse_totalRNA_kidney),mouse_totalRNA_kidney))


[(sum(human_liver),human_liver), (sum(human_kidney),human_kidney), (sum(cow_mRNA_liver),cow_mRNA_liver), (sum(cow_mRNA_kidney),cow_mRNA_kidney), (sum(cow_totalRNA_liver),cow_totalRNA_liver), (sum(cow_totalRNA_kidney),cow_totalRNA_kidney), (sum(mouse_mRNA_liver),mouse_mRNA_liver), (sum(mouse_mRNA_kidney),mouse_mRNA_kidney), (sum(mouse_totalRNA_liver),mouse_totalRNA_liver), (sum(mouse_totalRNA_kidney),mouse_totalRNA_kidney)]

In [28]:
val models = rename(modelSum, modelRenamings.toMap).cache

models = [uniref90: string, human_liver: double ... 9 more fields]


[uniref90: string, human_liver: double ... 9 more fields]

In [29]:
models.count

96855

In [31]:
%%dataframe models

uniref90,human_liver,human_kidney,cow_mRNA_liver,cow_mRNA_kidney,cow_totalRNA_liver,cow_totalRNA_kidney,mouse_mRNA_liver,mouse_mRNA_kidney,mouse_totalRNA_liver,mouse_totalRNA_kidney
UniRef90_A0A075B5M4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.273356,0.529034
UniRef90_A0A087WQS9,0.0,0.0,0.0,0.0,0.0,0.0,0.008303,0.018094,0.017657,0.031973
UniRef90_A0A087WRG2,0.534042,1.572286,0.0,0.0,0.0,0.0,0.704116,1.694746,1.25442,1.69502
UniRef90_A0A087WRS0,0.0,0.0,0.0,0.0,0.0,0.0,0.074889,0.724762,0.0,0.51444
UniRef90_A0A087WS80,0.0,0.0,0.0,0.0,0.0,0.0,0.125387,0.740652,0.0,13.04524
UniRef90_A0A087WU18,0.0,1.0E-6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A087WU77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A087WVF0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A087WWR9,0.0,0.115162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A087WWV1,0.020154,0.070128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
nonModelUniref90_outer.columns

[uniref90, gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney]

In [33]:
val allColumns = List("uniref90", "gray_whale_liver", "bowhead_whale_liver", "minke_liver", "NMR_liver", "human_liver", "mouse_totalRNA_liver", "mouse_mRNA_liver", "cow_totalRNA_liver", "cow_mRNA_liver", "gray_whale_kidney", "bowhead_whale_kidney", "minke_kidney", "NMR_kidney", "human_kidney", "mouse_totalRNA_kidney", "mouse_mRNA_kidney", "cow_totalRNA_kidney", "cow_mRNA_kidney" )

allColumns = List(uniref90, gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, human_liver, mouse_totalRNA_liver, mouse_mRNA_liver, cow_totalRNA_liver, cow_mRNA_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney, human_kidney, mouse_totalRNA_kidney, mouse_mRNA_kidney, cow_totalRNA_kidney, cow_mRNA_kidney)


List(uniref90, gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, human_liver, mouse_totalRNA_liver, mouse_mRNA_liver, cow_totalRNA_liver, cow_mRNA_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney, human_kidney, mouse_totalRNA_kidney, mouse_mRNA_kidney, cow_totalRNA_kidney, cow_mRNA_kidney)

In [34]:
val all =nonModelUniref90_outer.join(models, Seq("uniref90"), "outer").select(allColumns.head, allColumns.tail:_*).na.fill(0).sort($"gray_whale_liver".desc)

all = [uniref90: string, gray_whale_liver: double ... 17 more fields]


[uniref90: string, gray_whale_liver: double ... 17 more fields]

In [35]:
%%dataframe all

uniref90,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney
UniRef90_A0A1D3JP32,28912.07396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80702.3886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_B0JE22,23345.653565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44961.569793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_P0DMM1,13273.280865,2400.85,14073.428271,0.0,0.0,0.0,0.0,0.0,0.0,5.871845,25.693117,1321.29886,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_Q8IY31,10851.276487,0.0,31.497888,0.0,0.0,0.0,0.0,0.0,0.0,27763.924099,0.0,47.342378,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_Q95MP7,10473.906031,0.0,3043.48281,0.0,0.0,0.0,0.0,0.0,0.0,3570.825838,113.243136,2407.273068,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A286ZQC7,10060.120714,8386.28,24576.018837,0.0,0.0,0.0,0.0,0.0,0.0,1723.115534,46.086086,639.28565,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A212CAV9,6413.629848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12812.097645,1.358659,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A0E2JFH7,5342.438348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12792.727827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_L5MA95,3971.232224,871.505,3710.557181,0.0,0.0,0.0,0.0,0.0,0.0,3177.662877,67.772318,4141.129523,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A140GN66,3903.081981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1555.815324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
write(all.select(allColumns.head, allColumns.tail:_*), az(unirefPath + "/all_uniref90_outer.tsv"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/uniref90/all_uniref90_outer.tsv

Reactome
---------

Identifiers Mapping
--------------------

The mapping files consist of a tab-separated table that indicates which external protein, gene or small molecule identifiers in the source database were mapped to Reactome pathway and reaction annotations. Our goal with distributing three sets of files for each different identifier type is to provide these mapping files link the source database identifier to:

* the lowest level pathway diagram or subset of the pathway
* all level pathway diagrams (filename appended with 'All_Levels’),
* all reaction events.

The columns within the mappings files follow a similar format:

* Source database identifier, e.g. UniProt, ENSEMBL, NCBI Gene or ChEBI identifier
* Reactome Pathway Stable identifier
* URL
* Event (Pathway or Reaction) Name
* Evidence Code
* Species


In [36]:
val reactomeTable = readTSV(az(reactomePath + "/UniProt2Reactome.txt"), false).toDF("id", "reactome_id", "reactome_url", "reactome_title", "evidence_code", "pathway_species")

reactomeTable = [id: string, reactome_id: string ... 4 more fields]


[id: string, reactome_id: string ... 4 more fields]

In [37]:
val reactome = reactomeTable.select("id","reactome_url", "reactome_title", "pathway_species").distinct.cache

reactome = [id: string, reactome_url: string ... 2 more fields]


[id: string, reactome_url: string ... 2 more fields]

In [38]:
val df = all.na.fill("0").withColumn("id", cut_id($"uniref90"))

df = [uniref90: string, gray_whale_liver: double ... 18 more fields]


[uniref90: string, gray_whale_liver: double ... 18 more fields]

In [39]:
val values = df.columns.tail.filter(_!="id")

values = Array(gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, human_liver, mouse_totalRNA_liver, mouse_mRNA_liver, cow_totalRNA_liver, cow_mRNA_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney, human_kidney, mouse_totalRNA_kidney, mouse_mRNA_kidney, cow_totalRNA_kidney, cow_mRNA_kidney)


[gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, human_liver, mouse_totalRNA_liver, mouse_mRNA_liver, cow_totalRNA_liver, cow_mRNA_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney, human_kidney, mouse_totalRNA_kidney, mouse_mRNA_kidney, cow_totalRNA_kidney, cow_mRNA_kidney]

In [44]:
val pathways = reactome.join(df, "id").groupBy("reactome_url").sum(values:_*)

Name: Compile Error
Message: <console>:149: error: too many arguments for method count: ()Long
       val pathways = reactome.join(df, "id").groupBy("reactome_url").sum(values:_*).count(values:_*)
                                                                                          ^

StackTrace: 

In [48]:
val pathwayCounts = reactome.join(df, "id").groupBy("reactome_url").count().withColumnRenamed("count", "uniref90_count")

pathwayCounts = [reactome_url: string, uniref90_count: bigint]


[reactome_url: string, uniref90_count: bigint]

In [49]:
val renamings = pathways.columns.filter(_.startsWith("sum(")).map(v=>v->v.replace("sum(","").replace(")", ""))

renamings = Array((sum(gray_whale_liver),gray_whale_liver), (sum(bowhead_whale_liver),bowhead_whale_liver), (sum(minke_liver),minke_liver), (sum(NMR_liver),NMR_liver), (sum(human_liver),human_liver), (sum(mouse_totalRNA_liver),mouse_totalRNA_liver), (sum(mouse_mRNA_liver),mouse_mRNA_liver), (sum(cow_totalRNA_liver),cow_totalRNA_liver), (sum(cow_mRNA_liver),cow_mRNA_liver), (sum(gray_whale_kidney),gray_whale_kidney), (sum(bowhead_whale_kidney),bowhead_whale_kidney), (sum(minke_kidney),minke_kidney), (sum(NMR_kidney),NMR_kidney), (sum(human_kidney),human_kidney), (sum(mouse_totalRNA_kidney),mouse_totalRNA_kidney), (sum(mouse_mRNA_kidney),mouse_mRNA_kidney), (sum(cow_totalRNA_kidney),cow_totalRNA_kidney), (sum(cow_mRNA_kidney),cow_mRNA_kidney))


[(sum(gray_whale_liver),gray_whale_liver), (sum(bowhead_whale_liver),bowhead_whale_liver), (sum(minke_liver),minke_liver), (sum(NMR_liver),NMR_liver), (sum(human_liver),human_liver), (sum(mouse_totalRNA_liver),mouse_totalRNA_liver), (sum(mouse_mRNA_liver),mouse_mRNA_liver), (sum(cow_totalRNA_liver),cow_totalRNA_liver), (sum(cow_mRNA_liver),cow_mRNA_liver), (sum(gray_whale_kidney),gray_whale_kidney), (sum(bowhead_whale_kidney),bowhead_whale_kidney), (sum(minke_kidney),minke_kidney), (sum(NMR_kidney),NMR_kidney), (sum(human_kidney),human_kidney), (sum(mouse_totalRNA_kidney),mouse_totalRNA_kidney), (sum(mouse_mRNA_kidney),mouse_mRNA_kidney), (sum(cow_totalRNA_kidney),cow_totalRNA_kidney), (sum(cow_mRNA_kidney),cow_mRNA_kidney)]

In [50]:
val pathways_renamed = rename(pathways, renamings.toMap)
 .sort($"gray_whale_liver".desc).cache

pathways_renamed = [reactome_url: string, gray_whale_liver: double ... 17 more fields]


[reactome_url: string, gray_whale_liver: double ... 17 more fields]

In [65]:
val reactome_counts = reactome.groupBy("reactome_url").count().withColumnRenamed("count", "all_pathway_members_count")

reactome_counts = [reactome_url: string, all_pathway_members_count: bigint]


[reactome_url: string, all_pathway_members_count: bigint]

In [66]:
%%dataframe reactome_counts

reactome_url,all_pathway_members_count
https://reactome.org/PathwayBrowser/#/R-OSA-8949215,52
https://reactome.org/PathwayBrowser/#/R-BTA-5668541,68
https://reactome.org/PathwayBrowser/#/R-DME-114608,326
https://reactome.org/PathwayBrowser/#/R-XTR-70895,23
https://reactome.org/PathwayBrowser/#/R-DRE-5654695,36
https://reactome.org/PathwayBrowser/#/R-DME-425397,8
https://reactome.org/PathwayBrowser/#/R-HSA-5676594,17
https://reactome.org/PathwayBrowser/#/R-GGA-1250342,19
https://reactome.org/PathwayBrowser/#/R-MMU-8939242,18
https://reactome.org/PathwayBrowser/#/R-SSC-69601,87


In [69]:
val reactome_pathways = reactome.select("reactome_url", "reactome_title", "pathway_species").distinct.join(reactome_counts, "reactome_url").cache

reactome_pathways = [reactome_url: string, reactome_title: string ... 2 more fields]


[reactome_url: string, reactome_title: string ... 2 more fields]

In [52]:
(all.count, pathways.count,pathwayCounts.count, pathways_renamed.count)

(119429,7001,7001,7001)

In [55]:
val withCounts = pathwayCounts.join(pathways_renamed, "reactome_url").sort($"gray_whale_liver".desc).cache

withCounts = [reactome_url: string, uniref90_count: bigint ... 18 more fields]


[reactome_url: string, uniref90_count: bigint ... 18 more fields]

In [56]:
%%dataframe withCounts

reactome_url,uniref90_count,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney
https://reactome.org/PathwayBrowser/#/R-HSA-156902,88,19637.931238,7514.556700000001,17703.458897,21663.525935,18056.367420000024,174108.85328099816,60877.01633299961,323051.8819499993,329410.42027599947,21307.646262000002,3627.6083249999997,23939.159025,13208.959111999999,31793.936546999867,268547.9726200002,100875.11032099958,358744.86299999995,254749.9923690004
https://reactome.org/PathwayBrowser/#/R-HSA-72706,111,18388.370119999996,6996.8179,18708.172548000002,22430.506553999996,18182.25419700002,190741.66758899813,67397.19484499955,558076.200818996,707918.9307499938,18016.065963,2871.2132520000005,25847.752873000005,13816.322236999998,32375.033277999897,287451.1578780002,108512.0454279996,654210.881260999,566184.8675109937
https://reactome.org/PathwayBrowser/#/R-HSA-156827,110,18372.797392999997,6936.7880000000005,18488.162880000003,22430.506553999996,18219.996078000022,192422.41019699816,67519.97816899956,562947.158072996,711774.3317439937,17961.455723,2846.702263,25588.544480000004,13816.322236999998,32429.634711999897,289196.7124680002,108702.38247599959,659361.4237349989,569511.2117949937
https://reactome.org/PathwayBrowser/#/R-HSA-9010553,163,18360.779705,6986.213008006,18618.633012000006,23590.610610999996,18267.891083000028,194365.76055099815,68039.74809299955,565374.2222059959,716396.5449419938,17689.185163000002,2689.8907090000002,25317.394116,14920.502066999998,32394.969015999897,292241.05422100023,109576.9800089996,659777.6433139988,571765.7958729937
https://reactome.org/PathwayBrowser/#/R-HSA-72689,100,18246.386712,6847.500100000001,18130.938637000003,22430.506554000003,17598.789392000024,174488.42158999818,61079.5954419996,313608.75475699926,321039.9499599994,17450.452099000002,2688.6024390000002,24797.766375,13816.322236999998,31277.417861999867,269237.48373000027,101277.72807599955,344422.8744569999,250064.57739200036
https://reactome.org/PathwayBrowser/#/R-HSA-1799339,110,18113.539709,6777.214347000001,17925.240543000004,22038.591883,17613.580604000024,174391.7837369982,61028.73240299962,311462.58389199944,319400.0883849994,17137.423287999994,2647.101637,24231.530167000008,13804.67815,31214.86833499987,268733.7740940001,100978.22664899957,340236.83312999987,246015.42946900037
https://reactome.org/PathwayBrowser/#/R-HSA-6791226,178,17982.048524,6724.26063,17864.019323999997,22131.229646000003,17500.679940000013,174123.99484899812,60921.19005199958,311698.24178499926,319164.5997849995,17055.693660000004,2650.6627010000007,24215.529065000002,13790.582675,31166.74560799986,268432.4852600001,100828.90403499958,341034.35885200003,247001.49380800032
https://reactome.org/PathwayBrowser/#/R-HSA-2408557,92,17932.065565,6693.4741,17733.587465,21794.305088,17455.04999000002,173520.24906099815,60690.125081999606,309695.6900329993,315902.79541199945,16946.607499,2633.708691,24041.630452,13287.515135,31050.70096499987,267832.5077650002,100588.66525999957,337962.33941899997,243655.5941930004
https://reactome.org/PathwayBrowser/#/R-HSA-975957,114,17909.649218999995,6680.638577,17794.997821000004,22101.339096,17990.96254700002,190894.37177099814,66927.51132899956,559244.6365519961,706842.2076549936,16887.385712999996,2635.5403220000003,24118.725124000004,13451.886583999998,32119.150049999906,287279.5596090002,107700.76133799959,653181.8445009991,563558.1783169936
https://reactome.org/PathwayBrowser/#/R-HSA-72764,92,17906.438735,6690.2869200000005,17718.359936,21663.525935,17427.350567000023,173567.62225299815,60711.00439299961,309746.8264639994,316005.64865499944,16885.082876,2622.981641,23947.329723000003,13208.959111999999,31045.271758999865,267887.29741000023,100622.70700199956,338076.634431,243757.75488

In [97]:
%%dataframe pathways_renamed

reactome_url,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney
https://reactome.org/PathwayBrowser/#/R-HSA-156902,19637.931237999997,7514.556699999999,17703.458897,21663.525934999998,18056.367420000013,174108.85328099807,60877.01633299962,323051.8819499993,329410.4202759994,21307.646261999995,3627.6083250000006,23939.159025,13208.959112,31793.93654699987,268547.9726200002,100875.11032099958,358744.86299999995,254749.99236900033
https://reactome.org/PathwayBrowser/#/R-HSA-72706,18388.370119999996,6996.8179,18708.172548000002,22430.506553999996,18182.254197000013,190741.6675889981,67397.19484499958,558076.2008189958,707918.9307499935,18016.065963,2871.213252000001,25847.752873,13816.322236999998,32375.033277999883,287451.15787800035,108512.0454279996,654210.8812609988,566184.8675109937
https://reactome.org/PathwayBrowser/#/R-HSA-156827,18372.797393,6936.7880000000005,18488.162880000003,22430.506554,18219.99607800001,192422.4101969982,67519.97816899956,562947.1580729957,711774.3317439939,17961.455723000003,2846.702263,25588.54448,13816.322237000002,32429.634711999875,289196.7124680004,108702.38247599962,659361.4237349988,569511.2117949937
https://reactome.org/PathwayBrowser/#/R-HSA-9010553,18360.779705,6986.213008006,18618.633012000002,23590.610610999993,18267.891083000017,194365.76055099812,68039.74809299957,565374.2222059959,716396.5449419938,17689.185163,2689.8907090000002,25317.394116,14920.502066999998,32394.969015999883,292241.05422100023,109576.98000899961,659777.6433139985,571765.7958729938
https://reactome.org/PathwayBrowser/#/R-HSA-72689,18246.386712000003,6847.5001,18130.938637000003,22430.506554,17598.789392000017,174488.42158999815,61079.595441999605,313608.75475699926,321039.9499599994,17450.452099000002,2688.6024390000002,24797.766375,13816.322237000002,31277.417861999864,269237.4837300002,101277.72807599958,344422.874457,250064.57739200038
https://reactome.org/PathwayBrowser/#/R-HSA-1799339,18113.539709,6777.214346999998,17925.240543,22038.591882999997,17613.58060400002,174391.78373699813,61028.73240299962,311462.5838919993,319400.08838499937,17137.423288,2647.101637,24231.530167000008,13804.67815,31214.86833499987,268733.77409400017,100978.22664899958,340236.83312999987,246015.42946900037
https://reactome.org/PathwayBrowser/#/R-HSA-6791226,17982.048523999998,6724.26063,17864.019324000004,22131.229645999992,17500.679940000016,174123.99484899806,60921.19005199962,311698.24178499944,319164.5997849994,17055.69366,2650.6627010000007,24215.529065000002,13790.582674999998,31166.745607999877,268432.48526000016,100828.90403499959,341034.3588520001,247001.49380800035
https://reactome.org/PathwayBrowser/#/R-HSA-2408557,17932.065564999997,6693.474099999999,17733.587465,21794.305087999997,17455.049990000018,173520.2490609981,60690.12508199962,309695.6900329993,315902.7954119994,16946.607498999998,2633.7086910000007,24041.630452,13287.515135,31050.700964999876,267832.5077650002,100588.66525999959,337962.33941899997,243655.59419300032
https://reactome.org/PathwayBrowser/#/R-HSA-975957,17909.649219000003,6680.638577,17794.997821,22101.339096,17990.962547000014,190894.37177099808,66927.5113289996,559244.6365519959,706842.2076549937,16887.385713,2635.540322,24118.725124000004,13451.886584,32119.150049999884,287279.5596090004,107700.76133799962,653181.844500999,563558.1783169935
https://reactome.org/PathwayBrowser/#/R-HSA-192823,17906.438735000003,6675.6617,17703.458897,21663.525934999998,17424.269848000014,173618.9771419981,60728.45172599962,309762.04346499935,316014.44830399944,16885.082876,2622.981641,23939.159025,13208.959112,31035.423677999865,268042.1216740002,100640.11610999961,338045.212011,243744.44618900036


In [102]:
%%dataframe reactome

id,reactome_url,reactome_title,pathway_species
A0A075B5W3,https://reactome.org/PathwayBrowser/#/R-MMU-5690714,CD22 mediated BCR regulation,Mus musculus
A0A075B5X7,https://reactome.org/PathwayBrowser/#/R-MMU-173623,Classical antibody-mediated complement activation,Mus musculus
A0A075B5X7,https://reactome.org/PathwayBrowser/#/R-MMU-977606,Regulation of Complement cascade,Mus musculus
A0A075B678,https://reactome.org/PathwayBrowser/#/R-MMU-173623,Classical antibody-mediated complement activation,Mus musculus
A0A087WR31,https://reactome.org/PathwayBrowser/#/R-MMU-198933,Immunoregulatory interactions between a Lymphoid and a non-Lymphoid cell,Mus musculus
A0A087WSQ5,https://reactome.org/PathwayBrowser/#/R-MMU-877300,Interferon gamma signaling,Mus musculus
A0A096MIY3,https://reactome.org/PathwayBrowser/#/R-RNO-110357,Displacement of DNA glycosylase by APEX1,Rattus norvegicus
A0A096MJA8,https://reactome.org/PathwayBrowser/#/R-RNO-2565942,Regulation of PLK1 Activity at G2/M Transition,Rattus norvegicus
A0A096MJD1,https://reactome.org/PathwayBrowser/#/R-RNO-72187,mRNA 3'-end processing,Rattus norvegicus
A0A096MK10,https://reactome.org/PathwayBrowser/#/R-RNO-202733,Cell surface interactions at the vascular wall,Rattus norvegicus


In [104]:
(reactomeTable.count, reactome.count, reactome.select("id").distinct.count, reactome.select("reactome_url").distinct.count)

(456771,453594,153540,17448)

In [106]:
reactomeURLs.count

17448

In [71]:
val all_pathways = reactome_pathways.join(withCounts, "reactome_url").sort($"gray_whale_liver".desc).cache

all_pathways = [reactome_url: string, reactome_title: string ... 21 more fields]


[reactome_url: string, reactome_title: string ... 21 more fields]

In [72]:
all_pathways.count

7001

In [73]:
%%dataframe all_pathways

reactome_url,reactome_title,pathway_species,all_pathway_members_count,uniref90_count,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney
https://reactome.org/PathwayBrowser/#/R-HSA-156902,Peptide chain elongation,Homo sapiens,90,88,19637.931238,7514.556700000001,17703.458897,21663.525935,18056.367420000024,174108.85328099816,60877.01633299961,323051.8819499993,329410.42027599947,21307.646262000002,3627.6083249999997,23939.159025,13208.959111999999,31793.936546999867,268547.9726200002,100875.11032099958,358744.86299999995,254749.9923690004
https://reactome.org/PathwayBrowser/#/R-HSA-72706,GTP hydrolysis and joining of the 60S ribosomal subunit,Homo sapiens,113,111,18388.370119999996,6996.8179,18708.172548000002,22430.506553999996,18182.25419700002,190741.66758899813,67397.19484499955,558076.200818996,707918.9307499938,18016.065963,2871.2132520000005,25847.752873000005,13816.322236999998,32375.033277999897,287451.1578780002,108512.0454279996,654210.881260999,566184.8675109937
https://reactome.org/PathwayBrowser/#/R-HSA-156827,L13a-mediated translational silencing of Ceruloplasmin expression,Homo sapiens,112,110,18372.797392999997,6936.7880000000005,18488.162880000003,22430.506553999996,18219.996078000022,192422.41019699816,67519.97816899956,562947.158072996,711774.3317439937,17961.455723,2846.702263,25588.544480000004,13816.322236999998,32429.634711999897,289196.7124680002,108702.38247599959,659361.4237349989,569511.2117949937
https://reactome.org/PathwayBrowser/#/R-HSA-9010553,Regulation of expression of SLITs and ROBOs,Homo sapiens,172,163,18360.779705,6986.213008006,18618.633012000006,23590.610610999996,18267.891083000028,194365.76055099815,68039.74809299955,565374.2222059959,716396.5449419938,17689.185163000002,2689.8907090000002,25317.394116,14920.502066999998,32394.969015999897,292241.05422100023,109576.9800089996,659777.6433139988,571765.7958729937
https://reactome.org/PathwayBrowser/#/R-HSA-72689,Formation of a pool of free 40S subunits,Homo sapiens,102,100,18246.386712,6847.500100000001,18130.938637000003,22430.506554000003,17598.789392000024,174488.42158999818,61079.5954419996,313608.75475699926,321039.9499599994,17450.452099000002,2688.6024390000002,24797.766375,13816.322236999998,31277.417861999867,269237.48373000027,101277.72807599955,344422.8744569999,250064.57739200036
https://reactome.org/PathwayBrowser/#/R-HSA-1799339,SRP-dependent cotranslational protein targeting to membrane,Homo sapiens,113,110,18113.539709,6777.214347000001,17925.240543000004,22038.591883,17613.580604000024,174391.7837369982,61028.73240299962,311462.58389199944,319400.0883849994,17137.423287999994,2647.101637,24231.530167000008,13804.67815,31214.86833499987,268733.7740940001,100978.22664899957,340236.83312999987,246015.42946900037
https://reactome.org/PathwayBrowser/#/R-HSA-6791226,Major pathway of rRNA processing in the nucleolus and cytosol,Homo sapiens,183,178,17982.048524,6724.26063,17864.019323999997,22131.229646000003,17500.679940000013,174123.99484899812,60921.19005199958,311698.24178499926,319164.5997849995,17055.693660000004,2650.6627010000007,24215.529065000002,13790.582675,31166.74560799986,268432.4852600001,100828.90403499958,341034.35885200003,247001.49380800032
https://reactome.org/PathwayBrowser/#/R-HSA-2408557,Selenocysteine synthesis,Homo sapiens,94,92,17932.065565,6693.4741,17733.587465,21794.305088,17455.04999000002,173520.24906099815,60690.125081999606,309695.6900329993,315902.79541199945,16946.607499,2633.708691,24041.630452,13287.515135,31050.70096499987,267832.5077650002,100588.66525999957,337962.33941899997,243655.5941930004
https://reactome.org/PathwayBrowser/#/R-HSA-975957,Nonsense Mediated Decay (NMD) enhanced by the Exon Junction Complex (EJC),Homo sapiens,117,114,17909.64921

In [75]:
write(all_pathways, az(expressionsPath + "/pathways/all_pathways.tab"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/pathways/all_pathways.tab

In [111]:
all_pathways.count

7001

In [76]:
val bowhead = readTSV(az(unirefPath + "/bowhead_alaska_uniref90_all.tab"  ), true)

bowhead = [uniref90: string, transcript: string ... 13 more fields]


[uniref90: string, transcript: string ... 13 more fields]

In [1]:
bowhead.columns

[uniref90, transcript, type, go, uniprot_ac, taxon, uniprot_id, entrez, refSeq, gi, uniparc, pubmed, embl, sequence, liver]

In [80]:
%%dataframe bowhead.select("go")

go
null
GO:0034116; GO:0050839; GO:0007165; GO:0031639; GO:0045087; GO:0070062;GO:0072562; GO:0009897; GO:0005102; GO:0090277; GO:0051592; GO:0030168; GO:0072378; GO:0043152; GO:0050714; GO:0002250; GO:0005938; GO:0045921; GO:0070527; GO:1902042; GO:0070374; GO:0030674; GO:0007160; GO:0005577; GO:0042730;GO:0005577; GO:0051087; GO:0005198; GO:0007596; GO:0051258;GO:0005576; GO:2000352; GO:0045907; GO:0031091
GO:0005623; GO:0008199; GO:0006879; GO:0006826
GO:0060228; GO:0033700; GO:0032805; GO:0043407; GO:0002021; GO:0001937;GO:0042627; GO:0034362; GO:0007186; GO:1905855; GO:0010875; GO:0051000; GO:0045807; GO:0017127; GO:0042159; GO:0006874; GO:0048844; GO:0010468; GO:0032462; GO:0070062; GO:0042982; GO:0007616; GO:0005783; GO:0006707; GO:0008289; GO:0005794; GO:0043691; GO:0031012; GO:0043537; GO:0010873; GO:0006869; GO:0046889; GO:0007263; GO:0044794; GO:0034382; GO:0072562; GO:0060999; GO:1901628; GO:0010544; GO:0034447; GO:0097114; GO:0030828; GO:0050750; GO:1902995; GO:1902952; GO:0042803;GO:0106003; GO:0005886; GO:0051651; GO:1902430; GO:0090209; GO:1905860; GO:0005634; GO:1901630; GO:0042158; GO:0019068; GO:0034384; GO:0034374; GO:0090090; GO:0061771; GO:0034363; GO:0035641; GO:0001540; GO:2000822; GO:0015909; GO:0005543; GO:1900221; GO:0050728; GO:0097113; GO:0042157; GO:0033344; GO:0034364; GO:1905906; GO:0048156; GO:0017038; GO:0008201; GO:0010977; GO:0045541; GO:0042311; GO:0016209; GO:0006979; GO:1903002; GO:0034380; GO:0070326; GO:0034375; GO:1901215; GO:0055089; GO:0006641; GO:0019934; GO:0034361; GO:0034372; GO:0006898; GO:0070328; GO:1900272; GO:0046911; GO:0032489; GO:0042632; GO:0071813; GO:1905890; GO:0010877; GO:0051044; GO:0042627
GO:0007165; GO:0005102; GO:0030168; GO:0030674;GO:0005577; GO:0051258
GO:0070062;GO:0072562; GO:0031012; GO:0042803; GO:0005886; GO:0019862; GO:0004867; GO:0020037; GO:0036094;GO:0005576; GO:0018298
GO:0005319
GO:0005737; GO:0006526; GO:0000050; GO:0000053; GO:0004055; GO:0005524
GO:0015067;GO:0070062; GO:0014889; GO:0005739;GO:0015067; GO:0007275; GO:0006601; GO:0005758; GO:0007611; GO:0015068
GO:0005783;GO:0022625; GO:0006412; GO:0016020;GO:0015934; GO:0003723; GO:0002181; GO:0003735;GO:0005840
